In [7]:
import transformers
import json

import datasets
squad_dataset = datasets.load_dataset('squad')
from datasets import concatenate_datasets

import pandas as pd
import json
import numpy as np
covid_file = '../data/COVID-QA.json'
jsonfile = open(covid_file, 'r')

covidQA = jsonfile.read()
jsonfile.close()

covid_data = json.loads(covidQA)

06/26/2021 16:18:33 - WARNING - datasets.builder -   Reusing dataset squad (C:\Users\unitu\.cache\huggingface\datasets\squad\plain_text\1.0.0\6b6c4172d0119c74515f44ea0b8262efe4897f2ddb6613e5e915840fdc309c16)


In [8]:
from collections import OrderedDict

def convert_to_squad_format(data):
    covid_qa_squad_format = []
    for rows in data['data']:
      for context in rows['paragraphs']:
        for qa_pairs in context['qas']:

          features = {'answers': {'text':[qa_pairs['answers'][0]['text']],
                                  'answer_start': np.array([qa_pairs['answers'][0]['answer_start']], dtype=np.int32)},
                      'context':str(context['context']),
                      'id':str(context['document_id']),
                      'question':qa_pairs['question'],
                      'title': 'COVID_19'
                      }
          covid_qa_squad_format.append(features)
    covid_df = pd.DataFrame(covid_qa_squad_format)
    covid_dict = {'features', covid_qa_squad_format}
    return (datasets.Dataset.from_dict(covid_df))


def make_and_save_full_dataset(covid,squad, path):
    full_data = datasets.dataset_dict.DatasetDict({'covid':covid, 'squad':squad})
    full_data.save_to_disk(path)

def update_dataset_dict(to_remove_per_step, covid,squad, path):
    squad.shuffle()
    #covid.shuffle()
    if squad.num_rows > to_remove_per_step:
        #covid = datasets.Dataset.from_dict(covid[:])
        squad = datasets.Dataset.from_dict(squad[:-to_remove_per_step])
        make_and_save_full_dataset(covid,squad,path)

In [9]:
#covid_qa = convert_to_squad_format(covid_data)
covid_qa = datasets.load_dataset('custom_squad.py', data_files= {'train':covid_file})['train']
#covid_qa = covid_qa['train']
squad_qa = concatenate_datasets([squad_dataset['train'], squad_dataset['validation']])
covid_and_squad_dataset_path = "../data/squad_and_covidQA/"
#this is just for testing purposes, I am going to make both of these files very small only at max 3000 datasets
squad_qa = datasets.Dataset.from_dict(squad_qa[:100])
covid_qa = datasets.Dataset.from_dict(covid_qa[:50])

06/26/2021 16:18:33 - WARNING - datasets.builder -   Using custom data configuration default-38ab67c17eb3db2d
06/26/2021 16:18:33 - WARNING - datasets.builder -   Reusing dataset squad (C:\Users\unitu\.cache\huggingface\datasets\squad\default-38ab67c17eb3db2d\0.0.0\cb00e306c4924563ce3d1292a1ce1b86b2753dab6285ce43c87b39c5bda3ef4e)


In [10]:
make_and_save_full_dataset(covid_qa,squad_qa,covid_and_squad_dataset_path)

K = 3
to_remove_per_step = int(squad_qa.num_rows / K)

OSError: [Errno 22] Invalid argument: 'C:/Users/unitu/PycharmProjects/UCSC-NLP-Capstone/data/squad_and_covidQA/covid/dataset.arrow'

In [ ]:
print(covid_qa.num_rows)
squad_qa

In [ ]:
for i in range(K):
    print('**************************************************')
    print('==================================================')
    print('          At Gradual Fine Tuning Step: ',i+1)
    print('**************************************************')
    print('==================================================')
    if i < 1:
      %run run_qa.py \
      --model_name_or_path roberta-base \
      --dataset_name ../data/squad_and_covidQA/ \
      --do_train \
      --do_eval \
      --do_predict \
      --per_device_train_batch_size 35\
      --per_device_eval_batch_size 35\
      --evaluation_strategy "no" \
      --save_strategy "no" \
      --logging_strategy "epoch" \
      --learning_rate 3e-5 \
      --num_train_epochs 1 \
      --max_seq_length 384 \
      --doc_stride 128 \
      --k_fold_cross_valid 5 \
      --output_dir ../models/gradual_ft_baseline \
      --overwrite_output_dir
    else:
      %run run_qa.py \
      --model_name_or_path ../models/gradual_ft_baseline/Split-5/ \
      --dataset_name ../data/squad_and_covidQA/ \
      --do_train \
      --do_eval \
      --do_predict \
      --per_device_train_batch_size 35\
      --per_device_eval_batch_size 35\
      --evaluation_strategy "no" \
      --save_strategy "no" \
      --logging_strategy "epoch" \
      --learning_rate 3e-5 \
      --num_train_epochs 1 \
      --max_seq_length 384 \
      --doc_stride 128 \
      --k_fold_cross_valid 5 \
      --output_dir ../models/gradual_ft_baseline/ \
      --overwrite_output_dir
    if to_remove_per_step > squad_qa.num_rows:
        break
    covid_qa = datasets.Dataset.from_dict(covid_qa[:])
    squad_qa = datasets.Dataset.from_dict(squad_qa[:-to_remove_per_step])
    make_and_save_full_dataset(covid_qa,squad_qa,covid_and_squad_dataset_path)

print('Finished process')

In [ ]:
k_fold = 5
while to_remove_per_step < squad_qa.num_rows:
    print(covid_qa.num_rows)
    print(squad_qa.num_rows)
    cur_datasets = []
    for k in range(k_fold):
        cur_datasets.append(covid_qa.shard(num_shards=k_fold, index=k, contiguous=True))
    squad_qa.shuffle()
    covid_qa.shuffle()
    covid_qa = datasets.Dataset.from_dict(covid_qa[:])
    squad_qa = datasets.Dataset.from_dict(squad_qa[:-to_remove_per_step])
    make_and_save_full_dataset(covid_qa,squad_qa,covid_and_squad_dataset_path)